In [17]:
%reload_ext autoreload
%autoreload 2

import os
import sys

# Add the root folder to the module search path
# Get the current directory
current_directory = os.getcwd()

# Move two levels up (go to the parent directory of the parent directory)
two_levels_up_directory = os.path.dirname(os.path.dirname(current_directory))

print(two_levels_up_directory)

sys.path.append(two_levels_up_directory)

/Users/lingjiekong/Documents/github/cambioml/pykoi


In [18]:
!ngrok config add-authtoken xxx

Authtoken saved to configuration file: /Users/lingjiekong/Library/Application Support/ngrok/ngrok.yml


In [19]:
import pykoi

In [22]:
QA_CSV_HEADER_ID = 'ID'
QA_CSV_HEADER_QUESTION = 'Question'
QA_CSV_HEADER_ANSWER = 'Answer'
QA_CSV_HEADER_VOTE_STATUS = 'Vote Status'
QA_CSV_HEADER_TIMESTAMPS = 'Timestamp'
QA_CSV_HEADER = (
    QA_CSV_HEADER_ID,
    QA_CSV_HEADER_QUESTION,
    QA_CSV_HEADER_ANSWER,
    QA_CSV_HEADER_VOTE_STATUS,
    QA_CSV_HEADER_TIMESTAMPS
)

In [23]:
qa_database = pykoi.QuestionAnswerDatabase()

### Test Cambio App

In [24]:
model = pykoi.ModelFactory.create_model(
    model_name="huggingface", ## TODO: Change model_name to model_source
    pretrained_model_name_or_path="tiiuae/falcon-7b", ## TODO: Change pretrained_model_name_or_path to model_name
    trust_remote_code=True, ## TODO: set as default
    load_in_8bit=True)

[HuggingfaceModel] loading model...


A new version of the following files was downloaded from https://huggingface.co/tiiuae/falcon-7b:
- configuration_RW.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.
A new version of the following files was downloaded from https://huggingface.co/tiiuae/falcon-7b:
- modelling_RW.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


ImportError: Using `load_in_8bit=True` requires Accelerate: `pip install accelerate` and the latest version of bitsandbytes `pip install -i https://test.pypi.org/simple/ bitsandbytes` or pip install bitsandbytes` 

In [25]:
chatbot = pykoi.Chatbot(model=model, feedback="vote")

NameError: name 'model' is not defined

In [26]:
qa_dashboard = pykoi.Dashboard(database=qa_database)

In [27]:
app = pykoi.Application(debug=False, share=True)
app.add_component(chatbot)
app.add_component(qa_dashboard)
app.run()


NameError: name 'chatbot' is not defined

### Train RLHF using the data from database

In [28]:
# !pip install -q trl peft evaluate datasets pandas

In [29]:
my_data_pd = qa_database.retrieve_all_question_answers_as_pandas()
my_data_pd.head()

ValueError: Length mismatch: Expected axis has 0 elements, new values have 5 elements

In [ ]:
my_data_pd = my_data_pd[my_data_pd[QA_CSV_HEADER_VOTE_STATUS]=="up"]
my_data_pd.shape

NameError: name 'my_data_pd' is not defined

In [30]:
from datasets import Dataset

my_data_pd = my_data_pd[[QA_CSV_HEADER_ID,
                        QA_CSV_HEADER_QUESTION,
                        QA_CSV_HEADER_ANSWER]]
print("My local database has {} samples".format(my_data_pd.shape[0]))
dataset = Dataset.from_dict(my_data_pd)
dataset

NameError: name 'my_data_pd' is not defined

### Train with RLHF

In [31]:
# from pykoi.rlhf.rlhf import RLHFConfig, SFT ##, RewardTrainer, RL

config = pykoi.RLHFConfig(base_model_path="meta-llama/Llama-2-7b-hf", dataset_type="local_db")

rlhf_step1_sft = pykoi.SFT(config)

OSError: meta-llama/Llama-2-7b-hf is not a local folder and is not a valid model identifier listed on 'https://huggingface.co/models'
If this is a private repository, make sure to pass a token having permission to this repo with `use_auth_token` or log in with `huggingface-cli login` and pass `use_auth_token=True`.

In [32]:
rlhf_step1_sft.train_and_save("../../rlhf_tests/step1_07211538")

NameError: name 'rlhf_step1_sft' is not defined

### Insert my data to the DB

In [24]:
import pandas as pd
my_stackoverflow_dataset = pd.read_csv("/home/ubuntu/pykoi/pykoi/my_sql_data.csv", index_col=0)
my_stackoverflow_dataset.head(100)

,ID,Question,Answer,Vote Status,Timestamp
0,1,Who is Rachel?,CEO of CambioML,up,2023-07-20 23:19:25.202246
1,2,Who is Jared?,CTO of CambioML,up,2023-07-20 23:19:25.206972
2,3,Who is Kimi?,Nobody of CambioML,down,2023-07-20 23:19:25.210210
0,12891264,I am using jQuery fileupload plugin and I want...,"Looking at the library code, seems all events ...",NaN,NaN
1,588860,When reading about the Big Bang you’ll commonl...,No. The local energy *density* is reduced as t...,NaN,NaN
...,...,...,...,...,...
92,16617081,here is my object with sample data\n\n```\nObj...,"Firstly, if you've just got a date, I would su...",NaN,NaN
93,7690401,"Trying to add ""import"" statement to my new sca...",Make sure that you have a Java SDK configured ...,NaN,NaN
94,42071634,Is there any way to define a static method in ...,The simplest solution is probably to have the ...,NaN,NaN
95,42071634,Is there any way to define a static method in ...,The simplest solution is probably to have the ...,NaN,NaN


In [27]:
for row in my_stackoverflow_dataset.iloc[3:100].to_dict('records'):
    qa_id = qa_database.insert_question_answer(question=row[QA_CSV_HEADER_QUESTION],
                                       answer=row[QA_CSV_HEADER_ANSWER])
    qa_database.update_vote_status(id=qa_id, vote_status="up") #row[QA_CSV_HEADER_VOTE_STATUS])

In [ ]:
qa_database

In [28]:
# dataset = Dataset.from_dict(my_data_pd)